In [1]:
# Global imports
import biosteam as bst, thermosteam as tmo, biorefineries as bf
from biorefineries import cellulosic
from biosteam import main_flowsheet as F, units

In [2]:
# Local imports
from atj_saf.atj_bst.etj_chemicals import *
from atj_saf.atj_bst.etj_settings import feed_parameters, dehyd_data, olig_data, prod_selectivity, hydgn_data, price_data, h2_recovery
from atj_saf.atj_bst.etj_utils import calculate_ethanol_flow
from atj_saf.atj_bst.atj_bst_units import AdiabaticReactor, IsothermalReactor, EthanolStorageTank, HydrocarbonProductTank, HydrogenStorageTank, CatalystMixer
from atj_saf.atj_bst.atj_bst_tea_saf import ConventionalEthanolTEA
from cellulosic_tea_etj import create_cellulosic_ethanol_tea

In [3]:
bst.F.set_flowsheet('etj') # F is the main flowsheet


In [4]:
atj_chems = create_chemicals()
bst.settings.set_thermo(atj_chems) # Setting thermodynamic property pacakge for the chemicals


In [5]:
bst.settings.CEPCI = 800.8 # For the year 2023 from https://personalpages.manchester.ac.uk/staff/tom.rodgers/Interactive_graphs/CEPCI.html?reactors/CEPCI/index.html

In [6]:
etoh_flow = calculate_ethanol_flow(9)

In [7]:
# Bioethanol feed
etoh_in = bst.Stream(
    'Ethanol_In',
    Ethanol = etoh_flow,
    Water =  etoh_flow*((1-feed_parameters['purity'])/(feed_parameters['purity'])),
    units = 'kg/hr',
    T = feed_parameters['temperature'],
    P = feed_parameters['pressure'],
    phase = feed_parameters['phase'])




In [8]:
 # Reactions

#1) Gas phase dehydration of ethanol to ethylene 
dehydration_rxn = bst.Reaction('Ethanol,g -> Water,g + Ethylene,g', reactant = 'Ethanol', 
                            X = dehyd_data['conv'], phases = 'lg',  basis = 'mol')


#2) Ethylene oligomerization to olefins in gas and liquid phase
oligomerization_rxn = bst.ParallelReaction([
# Reaction definition                                     # Reactant                    # Conversion
bst.Reaction('2Ethylene,g -> Butene,g',            reactant = 'Ethylene',     X = olig_data['conv']*prod_selectivity['C4H8'],    basis = 'mol',  phases = 'lg',  correct_atomic_balance = True),
bst.Reaction('1.5Ethylene,g -> Hex-1-ene,g',       reactant = 'Ethylene',     X = olig_data['conv']*prod_selectivity['C6H12'],   basis = 'mol',  phases = 'lg',  correct_atomic_balance = True),
bst.Reaction('5Ethylene,g -> Dec-1-ene,l',         reactant = 'Ethylene',     X = olig_data['conv']*prod_selectivity['C10H20'],  basis = 'mol',  phases = 'lg',  correct_atomic_balance = True),
bst.Reaction('9Ethylene,g -> Octadec-1-ene,l',     reactant = 'Ethylene',     X = olig_data['conv']*prod_selectivity['C18H36'],  basis = 'mol',  phases = 'lg',  correct_atomic_balance = True)])


hydrogenation_rxn = bst.ParallelReaction([
# Reaction definition                                           # Reactant                    # Conversion
bst.Reaction('Butene,g + Hydrogen,g -> Butane,g',               reactant = 'Butene',          X = hydgn_data['conv'],  basis = 'mol',  phases = 'lg',  correct_atomic_balance = True),
bst.Reaction('Butene,l + Hydrogen,g -> Butane,l',               reactant = 'Butene',          X = hydgn_data['conv'],  basis = 'mol',  phases = 'lg',  correct_atomic_balance = True),
bst.Reaction('Hex-1-ene,g + Hydrogen,g -> Hexane,g',            reactant = 'Hex-1-ene',       X = hydgn_data['conv'],  basis = 'mol',  phases = 'lg',  correct_atomic_balance = True),
bst.Reaction('Hex-1-ene,l + Hydrogen,g -> Hexane,l',            reactant = 'Hex-1-ene',       X = hydgn_data['conv'],  basis = 'mol',  phases = 'lg',  correct_atomic_balance = True),
bst.Reaction('Dec-1-ene,l + Hydrogen,g -> Decane,l',            reactant = 'Dec-1-ene',       X = hydgn_data['conv'],  basis = 'mol',  phases = 'lg',  correct_atomic_balance = True),
bst.Reaction('Dec-1-ene,g + Hydrogen,g -> Decane,g',            reactant = 'Dec-1-ene',       X = hydgn_data['conv'],  basis = 'mol',  phases = 'lg',  correct_atomic_balance = True),
bst.Reaction('Octadec-1-ene,l + Hydrogen,g -> Octadecane,l',    reactant = 'Octadec-1-ene',   X = hydgn_data['conv'],  basis = 'mol',  phases = 'lg',  correct_atomic_balance = True),
bst.Reaction('Octadec-1-ene,g + Hydrogen,g -> Octadecane,g',    reactant = 'Octadec-1-ene',   X = hydgn_data['conv'],  basis = 'mol',  phases = 'lg',  correct_atomic_balance = True)])


In [9]:
# Recycle streams
dehyd_recycle = bst.MultiStream('dehyd_recycle', phases = ('g','l'))         # Unreacted ethanol
ethylene_recycle = bst.MultiStream('ethylene_recycle', phases = ('g','l'))   # Unreacted ethylene   
h2_recycle= bst.Stream(ID = 'h2_recycle', P = 3e6, phase = 'g')              # Excess hydrogen


In [10]:
etoh_storage = EthanolStorageTank(ins = etoh_in)
etoh_storage.simulate()

pump_1 = bst.Pump('PUMP1', ins = etoh_storage.outs[0], P = 1373000)    
pump_1.simulate()

furnace_1 = bst.HXutility('FURNACE_1', ins = pump_1.outs[0], T = 500, rigorous = True)
furnace_1.simulate()

mixer_1 = bst.Mixer('MIXER_1', ins = (furnace_1.outs[0], dehyd_recycle), rigorous = True)
mixer_1.simulate()

furnace_2 =  bst.HXutility('FURNACE_2', ins = mixer_1.outs[0], T = 481 + 273.15, rigorous = True)
furnace_2.simulate()

c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\biosteam\_unit.py:986: RuntimeWarning: the purchase cost item, 'Total Cost', has no defined bare-module factor in the 'EthanolStorageTank.F_BM' dictionary; bare-module factor now has a default value of 1
  warn(f"the purchase cost item, '{name}', has "


In [11]:
dehyd_1 = AdiabaticReactor('DEHYD_1', ins = furnace_2.outs[0],
                        conversion = dehyd_data['conv'],
                        temperature = dehyd_data['temp'],
                        pressure = dehyd_data['pressure'],
                        WHSV = dehyd_data['whsv'],
                        vessel_type = 'Vertical',
                        vessel_material = 'Stainless steel 316',
                        catalyst_price=price_data['dehydration_catalyst'],
                        catalyst_lifetime = dehyd_data['catalyst_lifetime'],
                        reaction = dehydration_rxn)
dehyd_1.simulate()

In [12]:
# Dehydration catalyst replacement 
syndol_wt = dehyd_1.get_design_result('Catalyst Weight', 'kg')
syndol_req = syndol_wt/dehyd_data['catalyst_lifetime']
syndol_replacement = bst.Stream('Dehyd_cat_replacement', Syndol = syndol_req, phase = 's', units = 'kg/yr' )


In [12]:
splitter_1 = bst.Splitter(ins = dehyd_1.outs[0], outs = ('flash_in', dehyd_recycle), split = 1)
splitter_1.simulate()

flash_1 = bst.Flash('FLASH_1', ins = splitter_1.outs[0], outs = ('ETHYLENE_WATER', 'WW_1'), T= 420,  P = 1.063e6)
flash_1.simulate()


comp_1 = bst.IsentropicCompressor('COMP_1', ins = flash_1.outs[0], P = 2e6, vle = True, eta = 0.72, driver_efficiency = 1)
comp_1.simulate()   

distillation_1 = bst.BinaryDistillation('DISTILLATION_1', ins = comp_1.outs[0], 
                                            outs = ('ethylene_water', 'WW'),
                                LHK = ('Ethylene', 'Water'), 
                                P = 2e+06,
                                y_top = 0.999, x_bot = 0.001, k = 2,
                                is_divided = False)
# distillation_1.check_LHK = False   # Does not check for volatile components that might show up in lights
distillation_1.simulate()

comp_2 = bst.IsentropicCompressor('COMP_2', ins = distillation_1.outs[0], P = olig_data['pressure'], vle = True, eta = 0.72, driver_efficiency = 1)
comp_2.simulate()

distillation_2 = bst.BinaryDistillation('DISTILLATION_2', ins = comp_2.outs[0],
                                LHK = ('Ethylene', 'Ethanol'),
                                P = 3.5e+06,
                                y_top = 0.9999, x_bot = 0.0001, k = 2,
                                is_divided = False)
distillation_2.simulate()

splitter_2 = bst.Splitter('SPLIT2', ins = distillation_1.outs[1], split = 0.6)
splitter_2.simulate()


cooler_3 = bst.HXutility('COOLER_3', ins = distillation_2.outs[0], T = 393.15, rigorous = True)
cooler_3.simulate()

mixer_2 = bst.Mixer(ID = 'MIXER_3', ins = (cooler_3.outs[0],ethylene_recycle), rigorous = True)
mixer_2.simulate()


c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\thermosteam\equilibrium\bubble_point.py:128: RuntimeWarning: Ethylene has no defined Dortmund groups; functional group interactions are ignored
  self.gamma = thermo.Gamma(chemicals)
c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\biosteam\units\design_tools\pressure_vessel.py:104: CostWarning: <Flash: FLASH_1> Vertical vessel weight (860.9 lb) is out of bounds (4200 to 1e+06 lb) for cost correlation
  return method(pressure, diameter, length)
c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\biosteam\units\design_tools\pressure_vessel.py:104: CostWarning: <Flash: FLASH_1> Vertical vessel length (6.5 ft) is out of bounds (12 to 40 ft) for cost correlation
  return method(pressure, diameter, length)
c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\biosteam\_unit.py:1241: CostWarning: <IsentropicCompressor: COMP_2> power (124.1 hp) is out of bounds (200 to 3e+04 hp) for cost correlation
  self._cost(**cost_kwargs) i

In [14]:
olig_1 = IsothermalReactor('OLIG_1', ins = mixer_2.outs[0],
                            conversion = olig_data['conv'],
                            temperature = olig_data['temp'],
                            pressure = olig_data['pressure'],
                            WHSV = olig_data['whsv'],
                            catalyst_price = price_data['oligomerization_catalyst'],
                        reaction = oligomerization_rxn)
olig_1.simulate()
# Oligomerization catalyst replacement stream


In [15]:
# Oligomerization catalyst replacement 
ni_si_al_wt = olig_1.get_design_result('Catalyst Weight', 'kg')
ni_si_al_req = ni_si_al_wt/olig_data['catalyst_lifetime']
ni_si_al_replacement = bst.Stream('Olig_cat_replacement', Nickel_SiAl = ni_si_al_req, phase = 's', units = 'kg/yr' )

In [16]:
((olig_1.outs[0].imol['Butene'] + olig_1.outs[0].imol['Hex-1-ene']
                    + olig_1.outs[0].imol['Dec-1-ene'] + olig_1.outs[0].imol['Octadec-1-ene']))+13.47506929898195

507.3201885922134

In [17]:
0.85*((olig_1.outs[0].imol['Butene'] + olig_1.outs[0].imol['Hex-1-ene']
                    + olig_1.outs[0].imol['Dec-1-ene'] + olig_1.outs[0].imol['Octadec-1-ene']))
    

419.76835139924674

In [18]:
splitter_3 = bst.Splitter('SPLIITER_3', ins = olig_1.outs[0], outs = (ethylene_recycle,'oligs'),  split = {'Ethylene':1.0})
splitter_3.simulate()

h2_in = bst.Stream(ID = 'Hydrogen_In',  P = 3e6, phase= 'g')

#mixer_3 = bst.Mixer('mix_try', ins = (h2_in, h2_recycle), rigorous = True)
#@mixer_3.add_specification(run = True)
#def h2_flow():
#    h2_flow = 3*((olig_1.outs[0].imol['Butene'] + olig_1.outs[0].imol['Hex-1-ene']
#                    + olig_1.outs[0].imol['Dec-1-ene'] + olig_1.outs[0].imol['Octadec-1-ene']))
#    
#    h2_in.imol['Hydrogen'] = h2_flow - h2_recycle.imol['Hydrogen']
#mixer_3.simulate()

h2_storage = HydrogenStorageTank('H2_STORAGE',ins = h2_in)
@h2_storage.add_specification(run = True)
def h2_flow():
    h2_flow = 3*((olig_1.outs[0].imol['Butene'] + olig_1.outs[0].imol['Hex-1-ene']
                    + olig_1.outs[0].imol['Dec-1-ene'] + olig_1.outs[0].imol['Octadec-1-ene']))
    
    h2_storage.ins[0].imol['Hydrogen'] = h2_flow - h2_recycle.imol['Hydrogen']
h2_storage.simulate()


mixer_4 = bst.Mixer(ins = (h2_storage.outs[0], splitter_3.outs[1], h2_recycle), rigorous = True)
mixer_4.simulate()


furnace_3 = bst.HXutility('FURNACE_3', mixer_4.outs[0], T = 350 +273.15, rigorous = True)
furnace_3.simulate()



c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\biosteam\_unit.py:986: RuntimeWarning: the purchase cost item, 'Total Cost', has no defined bare-module factor in the 'HydrogenStorageTank.F_BM' dictionary; bare-module factor now has a default value of 1
  warn(f"the purchase cost item, '{name}', has "
c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\thermosteam\equilibrium\bubble_point.py:128: RuntimeWarning: Hydrogen has no defined Dortmund groups; functional group interactions are ignored
  self.gamma = thermo.Gamma(chemicals)


In [19]:
hydgn_1 = AdiabaticReactor('hydgn', ins = furnace_3.outs[0],
                        conversion = hydgn_data['conv'],
                        temperature = hydgn_data['temp'],
                        pressure = hydgn_data['pressure'],
                        WHSV = hydgn_data['whsv'],
                        catalyst_price = price_data['hydrogenation_catalyst'],
                        reaction = hydrogenation_rxn)
hydgn_1.simulate()

In [20]:
# Hydrogenation catalyst replacement 
co_mo_wt = hydgn_1.get_design_result('Catalyst Weight', 'kg')
co_mo_req = co_mo_wt/hydgn_data['catalyst_lifetime']
co_mo_replacement = bst.Stream('Hydgn_cat_replacement', CobaltMolybdenum = co_mo_req, phase = 's', units = 'kg/yr' )

In [21]:
cooler_5 = bst.HXutility('COOLER_5', ins = hydgn_1.outs[0], T = 250, rigorous = True) 
cooler_5.simulate()


flash_2 = bst.Flash(ins = cooler_5-0, T = 250, P = 5e5)
flash_2.simulate()

psa_splitter = bst.Splitter('PSA_SPLITTER', ins = flash_2-0, outs = (h2_recycle,'BT_feed'),  split = {'Hydrogen':h2_recovery})


distillation_3 = bst.BinaryDistillation('DISTILLATION_3', ins = flash_2.outs[1],
                                outs = ('distillate', 'bottoms'),
                                LHK = ('Hexane', 'Decane'),
                                y_top = 0.99, x_bot = 0.01, k = 2,
                                is_divided = True)
distillation_3.simulate()

distillation_4 = bst.BinaryDistillation('DISTILLATION_4', ins = distillation_3.outs[1],
                                outs = ('distillate_1', 'bottoms_1'),
                                LHK = ('Decane', 'Octadecane'),
                                y_top = 0.99, x_bot = 0.01, k = 2,
                                is_divided = True)
distillation_4.simulate()

cooler_6 = bst.HXutility('COOLER_6', ins = distillation_3.outs[0]
                            ,V = 0, rigorous = True)
cooler_6.simulate()


cooler_7 = bst.HXutility('COOLER_7', ins = distillation_4.outs[0],T = 15+273.15, rigorous = True)
cooler_7.simulate()
cooler_7.outs[0].phase = 'l'   # Just setting it as liquid, because rigorous = True gives both l and g phases. 
                               # Actual phase is liquid only so asserting phase like this is fine

cooler_8 = bst.HXutility('COOLER_8', ins = distillation_4.outs[1],T = 15+273.15, rigorous = True)
cooler_8.simulate()
cooler_8.outs[0].phase = 'l'


rn_storage = HydrocarbonProductTank('RN_STORAGE', ins = cooler_6.outs[0], outs = 'RN')
rn_storage.simulate()

saf_storage = HydrocarbonProductTank('SAF_STORAGE', ins = cooler_7.outs[0], outs = 'SAF')
saf_storage.simulate()


rd_storage = HydrocarbonProductTank('RD_STORAGE', ins = cooler_8.outs[0], outs = 'RD')
rd_storage.simulate()




WW_mixer = bst.Mixer(ins = (flash_1-1, distillation_2-1, splitter_2-0, splitter_2-1), rigorous = True)
WW_mixer.simulate()

WW_cooler = bst.HXutility(ins = WW_mixer.outs[0], V = 0, rigorous = True)
WW_cooler.simulate()

WWT = bst.create_conventional_wastewater_treatment_system(ins=WW_cooler.outs[0])

BT = bst.facilities.BoilerTurbogenerator(fuel_price = price_data['NG'])   # Boiler turbogenerator
BT.ins[1] = F.BT_feed


catalyst_replacement_unit = CatalystMixer(ins = (syndol_replacement, ni_si_al_replacement, co_mo_replacement))
catalyst_replacement_unit.simulate()


c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\thermosteam\equilibrium\dew_point.py:129: RuntimeWarning: Hydrogen has no defined Dortmund groups; functional group interactions are ignored
  self.gamma = thermo.Gamma(chemicals)
c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\biosteam\units\design_tools\pressure_vessel.py:104: CostWarning: <RefluxDrum: reflux_drum> Vertical vessel weight (3435 lb) is out of bounds (4200 to 1e+06 lb) for cost correlation
  return method(pressure, diameter, length)
c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\biosteam\units\design_tools\pressure_vessel.py:104: CostWarning: <RefluxDrum: reflux_drum> Vertical vessel length (7.5 ft) is out of bounds (12 to 40 ft) for cost correlation
  return method(pressure, diameter, length)
c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\biosteam\units\design_tools\pressure_vessel.py:104: CostWarning: <RefluxDrum: reflux_drum> Vertical vessel weight (4156 lb) is out of bounds (4200 to 1e+06 

In [22]:
atj_sys = bst.System('atj_sys', path = (etoh_storage, pump_1, furnace_1, mixer_1, furnace_2, dehyd_1, splitter_1, flash_1, comp_1, 
                                        distillation_1, comp_2, distillation_2, splitter_2, cooler_3, mixer_2,
                                        olig_1, splitter_3, h2_storage, mixer_4, furnace_3, hydgn_1, cooler_5, 
                                        flash_2, psa_splitter, distillation_3, distillation_4, cooler_6, cooler_7, cooler_8,
                                        rn_storage, saf_storage, rd_storage, WW_mixer, WW_cooler, catalyst_replacement_unit), 
                                        facilities = [WWT, BT],
                                        recycle = (dehyd_recycle, ethylene_recycle, h2_recycle))
atj_sys.simulate()

In [23]:
atj_sys.show()

System: atj_sys
Highest convergence error among components in recycle
streams {S1-1, SPLIITER_3-0, PSA_SPLITTER-0} after 2 loops:
- flow rate   4.11e-02 kmol/hr (0.0049%)
- temperature 0.00e+00 K (0%)
ins...
[0] Ethanol_In  
    phase: 'l', T: 293.15 K, P: 101325 Pa
    flow (kmol/hr): Water    23.2
                    Ethanol  1.81e+03
[1] Hydrogen_In  
    phase: 'g', T: 298.15 K, P: 3e+06 Pa
    flow (kmol/hr): Hydrogen  647
[2] Dehyd_cat_replacement  
    phase: 's', T: 298.15 K, P: 101325 Pa
    flow (kmol/hr): Syndol  15.9
[3] Olig_cat_replacement  
    phase: 's', T: 298.15 K, P: 101325 Pa
    flow (kmol/hr): Nickel_SiAl  3.83
[4] Hydgn_cat_replacement  
    phase: 's', T: 298.15 K, P: 101325 Pa
    flow (kmol/hr): CobaltMolybdenum  0.671
[5] air_lagoon  
    phase: 'g', T: 298.15 K, P: 101325 Pa
    flow (kmol/hr): N2  10.9
                    O2  2.69
[6] caustic  
    phase: 'l', T: 298.15 K, P: 101325 Pa
    flow (kmol/hr): Water  10.5
                    NaOH   4.72
[7] -  

In [24]:
atj_sys.diagram(format = 'png', file = 'atj_s')

In [25]:
bst.nbtutorial()

In [26]:
catalyst_replacement_unit

CatalystMixer: M3
ins...
[0] Dehyd_cat_replacement  
    phase: 's', T: 298.15 K, P: 101325 Pa
    flow (kmol/hr): Syndol  15.9
[1] Olig_cat_replacement  
    phase: 's', T: 298.15 K, P: 101325 Pa
    flow (kmol/hr): Nickel_SiAl  3.83
[2] Hydgn_cat_replacement  
    phase: 's', T: 298.15 K, P: 101325 Pa
    flow (kmol/hr): CobaltMolybdenum  0.671
outs...
[0] s45  
    phase: 's', T: 298.15 K, P: 101325 Pa
    flow (kmol/hr): Syndol            15.9
                    Nickel_SiAl       3.83
                    CobaltMolybdenum  0.671


In [27]:
atj_sys.diagram()

In [28]:
# Mass balance error check
feed_flow = 0
for i in range(13):
    feed_flow += (atj_sys.feeds[i].F_mass)

prod_flow = 0
for i in range(12):
    prod_flow += (atj_sys.products[i].F_mass)

mass_bal_error = abs(feed_flow - prod_flow)

print(f'The feed flow rate is {round(feed_flow,2)} kg/hr and the product flow rate is {round(prod_flow,2)} kg/hr')
print(f'Difference is {mass_bal_error}')

The feed flow rate is 628565.71 kg/hr and the product flow rate is 628565.34 kg/hr
Difference is 0.3679155708523467


### Estimation of labor costs
In accordance with methodology in Sieder book

In [29]:

operators_per_section = 1  # operators per section from Seider recommendation
num_process_sections = 5  # number of proces sections from Seider recommendation [3 reactor, 2 separation]
num_operators_per_shift = operators_per_section * num_process_sections * 1  # multiplied by 2 for large continuous flow process (e.g., 1000 ton/day product). from Seider pg 505
num_shifts = 5  # number of shifts
pay_rate = 40  # $/hr
DWandB = num_operators_per_shift * num_shifts * 2080 * pay_rate  # direct wages and benefits. DWandB [$/year] = (operators/shift)*(5 shifts)*(40 hr/week)*(operating days/year-operator)*($/hr)
Dsalaries_benefits = 0.15 * DWandB  # direct salaries and benefits from Seider
O_supplies = 0.06 * DWandB  # Operating supplies and services from Seider
technical_assistance = 5 * 75000  # $/year. Technical assistance to manufacturing. assume 5 workers at $75000/year
control_lab = 5 * 80000  # $/year. Control laboratory. assume 5 workers at $80000/year
labor = DWandB + Dsalaries_benefits + O_supplies + technical_assistance + control_lab 

    

### Prices data 

In [30]:
F.Ethanol_In.price = price_data['ethanol']             # Ethanol price, subject to change once integration with Kays model happens
F.Hydrogen_In.price = price_data['hydrogen']
F.RN.price = price_data['renewable_naphtha']
saf_stream = F.SAF
F.RD.price = price_data['renewable_diesel']
F.Dehyd_cat_replacement.price = price_data['dehydration_catalyst']
F.Olig_cat_replacement.price = price_data['oligomerization_catalyst']
F.Hydgn_cat_replacement.price = price_data['hydrogenation_catalyst']


AttributeError: no registered item 'Ethanol_In'

In [ ]:
final_tea = create_cellulosic_ethanol_tea(atj_sys)


In [ ]:
final_tea.labor_cost = labor

In [ ]:
print(f'The Total Capital Investement for ETJ biorefinery is {round(final_tea.TCI/1e6,2)} MM USD')

The Total Capital Investement for ETJ biorefinery is 614.54 MM USD


In [ ]:
print(f'The MSP for ETJ-derived SAF is  {round(((final_tea.solve_price(F.SAF)*F.SAF.rho)/264.172),2)} USD/gal')

The MSP for ETJ-derived SAF is  7.03 USD/gal


In [ ]:
print('ETJ system installed equipment cost is', round(atj_sys.installed_equipment_cost / 1e6, 2), 'MM USD')
print('ETJ system purchase equipment cost is', round(atj_sys.purchase_cost / 1e6, 2), 'MM USD')


ETJ system installed equipment cost is 356.23 MM USD
ETJ system purchase equipment cost is 295.92 MM USD


In [ ]:
print(f'The Wastewater system cost is {round(F.WWTC.installed_cost/1e6,2)} MM USD')


The Wastewater system cost is 16.55 MM USD
